In [37]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [38]:
wd_path = r'C:\Users\13473\Desktop\e4040-2020Fall-Project-WNZH-jr4001-xy2419-gl2664'
os.chdir(wd_path)
os.getcwd()

'C:\\Users\\13473\\Desktop\\e4040-2020Fall-Project-WNZH-jr4001-xy2419-gl2664'

In [39]:
file_name = 'complete_universe_cleaned_normalized_jonn.csv'

In [40]:
import pandas as pd
import numpy as np

from datetime import datetime

In [5]:
df = pd.read_csv(os.path.join(wd_path,file_name),dtype={'COMNAM':str,'NCUSIP':str,'PERMNO':str,'PERMCO':str,'SICCD':str,'TICKER_PERMNO':str},low_memory=False,index_col=0)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
df['DATE'] = pd.to_datetime(df['DATE'],format='%Y-%m-%d')

In [7]:
sector_map = {'0':'Agriculture, Forestry and Fishing','10':'Mining','11':'Mining','12':'Mining','13':'Mining','14':'Mining','15':'Construction','16':'Construction','17':'Construction','18':'not used','19':'not used','20':'Manufacturing','21':'Manufacturing','22':'Manufacturing','23':'Manufacturing','24':'Manufacturing','25':'Manufacturing','26':'Manufacturing','27':'Manufacturing','28':'Manufacturing','29':'Manufacturing','30':'Manufacturing','31':'Manufacturing','32':'Manufacturing','33':'Manufacturing','34':'Manufacturing','35':'Manufacturing','36':'Manufacturing','37':'Manufacturing','38':'Manufacturing','39':'Manufacturing','40':'Transportation, Communications, Electric, Gas and Sanitary service','41':'Transportation, Communications, Electric, Gas and Sanitary service','42':'Transportation, Communications, Electric, Gas and Sanitary service','44':'Transportation, Communications, Electric, Gas and Sanitary service','45':'Transportation, Communications, Electric, Gas and Sanitary service','46':'Transportation, Communications, Electric, Gas and Sanitary service','47':'Transportation, Communications, Electric, Gas and Sanitary service','48':'Transportation, Communications, Electric, Gas and Sanitary service','49':'Transportation, Communications, Electric, Gas and Sanitary service','50':'Wholesale Trade','51':'Wholesale Trade','52':'Retail Trade','53':'Retail Trade','54':'Retail Trade','55':'Retail Trade','56':'Retail Trade','57':'Retail Trade','58':'Retail Trade','59':'Retail Trade','60':'Finance, Insurance','61':'Finance, Insurance','62':'Finance, Insurance','63':'Finance, Insurance','64':'Finance, Insurance','65':'Real Estate','66':'Finance, Insurance','67':'Finance, Insurance','70':'Services','71':'Services','72':'Services','73':'Services','74':'Services','75':'Services','76':'Services','78':'Services','79':'Services','80':'Services','81':'Services','82':'Services','83':'Services','86':'Services','87':'Services','89':'Services','91':'Public Administration','92':'Public Administration','94':'Public Administration','95':'Public Administration','96':'Public Administration','97':'Public Administration','99':'Nonclassifiable'}

In [8]:
temp = pd.get_dummies(df['SICCD'].str[:2].replace(sector_map))
df = pd.concat([df, temp], axis=1, sort=False)

In [41]:
import gc

In [42]:
del temp 
gc.collect()
temp = None

In [43]:
df['m+1_returns'] = df['m+1']
df.loc[:,'m+1'] = df.groupby(['DATE'])['m+1'].transform(lambda x: x.rank(pct = True))

In [44]:
identifier_cols = ['DATE','COMNAM','NCUSIP','PERMNO','PERMCO','SICCD','TICKER_PERMNO']
cross_sectional_cols=['accru', 'adv_sale', 'aftret_eq', 'aftret_equity','aftret_invcapx', 'at_tu', 'bm', 'CAPEI', 'capital_rat', 'cash_debt',
       'cash_lt', 'cash_rat', 'cfm', 'curr_debt', 'curr_rat', 'de_rat','debt_assets', 'debt_at', 'debt_capit', 'debt_ebitd', 'debt_invcap',
       'divyield', 'dltt_be', 'equity_invcap', 'evm', 'gpm', 'GProf', 'intcov','intcov_rat', 'invt_act', 'lt_debt', 'lt_ppent', 'npm', 'ocf_lct',
       'opmad', 'opmbd', 'pay_tu', 'pcf', 'pe_ex', 'pe_inc', 'pretret_earnat','pretret', 'profit_lct', 'ps', 'ptb', 'ptpm', 'quick_rat', 'rd_sale',
       'rect_act', 'rect_tu', 'roa', 'roce', 'roe', 'sale_equity','sale_invcap', 'short_debt', 'staff_sale', 'totdebt_invcap']
sector_cols = ['Agriculture, Forestry and Fishing', 'Construction','Finance, Insurance', 'Manufacturing', 'Mining', 'Nonclassifiable',
       'Public Administration', 'Real Estate', 'Retail Trade', 'Services','Transportation, Communications, Electric, Gas and Sanitary service',
       'Wholesale Trade', 'not used']
time_series_cols= ['m-12','m-11', 'm-10', 'm-9', 'm-8', 'm-7', 'm-6', 'm-5', 'm-4', 'm-3', 'm-2','m-1']
all_cols = cross_sectional_cols + sector_cols + time_series_cols
Y = ['m+1']

In [45]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Concatenate, Input, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau

In [46]:
'''Hybrid Regression Model'''
time_series_inputs = Input(shape=(12,1))
cross_sectional_inputs = Input(shape=(83,))

h1_rets=LSTM(units=50, return_sequences=True)(time_series_inputs)
time_series_output=LSTM(units=30, return_sequences=False)(h1_rets) 

combined_features = Concatenate()([cross_sectional_inputs, time_series_output])

h1=Dense(128, activation='relu')(combined_features)
h2=Dense(64, activation='relu')(h1)
h3=Dense(32, activation='relu')(h2)
outputs=Dense(1, activation='sigmoid')(h3)

hybrid_model=tf.keras.Model(inputs=[cross_sectional_inputs,time_series_inputs], outputs=outputs)

opt=tf.keras.optimizers.Adam(
    learning_rate=0.005,
)
hybrid_model.compile(
    optimizer=opt, loss='binary_crossentropy',metrics=['binary_crossentropy','mean_squared_error','mean_absolute_error','mean_absolute_percentage_error']
)
sample_model = tf.keras.models.load_model(os.path.join(wd_path,'models\hybrid_reg_model_train_period_12_1970_12_1980.hdf5'))

In [49]:
mask_test=(df[['DATE']]=='1981-01-31')
x = [df[all_cols].values,df[time_series_cols].values]
y = df[Y].values[:,0]

x_test, y_test= [i[mask_test['DATE'],:] for i in x], y[mask_test['DATE']]

In [50]:
sample_model.predict(x_test)

UnknownError:    Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[functional_18/lstm_10/PartitionedCall]] [Op:__inference_predict_function_14923]

Function call stack:
predict_function -> predict_function -> predict_function
